In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW
from tqdm import tqdm
import pandas as pd
import re
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd

# Load the data from the CSV file
data_url = "/content/MT_ar_faq_covidbert.csv"  # Replace with the actual path to your CSV file
faq_data = pd.read_csv(data_url)

# Display the first few rows of the dataframe
print(faq_data.head())


   Unnamed: 0                                           question  \
0           0                       What is a novel coronavirus?   
1           1  Why is the disease being called coronavirus di...   
2           2  Why might someone blame or avoid individuals a...   
3           3  How can people help stop stigma related to COV...   
4           4                   What is the source of the virus?   

                                              answer  \
0  A novel coronavirus is a new coronavirus that ...   
1  On February 11, 2020 the World Health Organiza...   
2  People in the U.S. may be worried or anxious a...   
3  People can fight stigma and help, not hurt, ot...   
4  Coronaviruses are a large family of viruses. S...   

                                         answer_html  \
0  <p>A novel coronavirus is a new coronavirus th...   
1  <p>On February 11, 2020 the World Health Organ...   
2  <p>People in the U.S. may be worried or anxiou...   
3  <p>People can fight stigma 

In [ ]:
import re
from sklearn.model_selection import train_test_split

# Function to clean the text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Remove @mentions and hashtags
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text.strip()

# Apply cleaning function
faq_data['cleaned_question'] = faq_data['question'].apply(clean_text)
faq_data['cleaned_answer'] = faq_data['answer'].apply(clean_text)

# Split the data into training and testing sets
train_data, test_data = train_test_split(faq_data, test_size=0.2, random_state=42)

print(train_data[['cleaned_question', 'cleaned_answer']].head())


                                      cleaned_question  \
79   how should healthcare personnel protect themse...   
161                          what about pregnant women   
109  if we dismiss school what do we need to consid...   
127   how to put on use take off and dispose of a mask   
95   what ppe should be worn by hcp providing care ...   

                                        cleaned_answer  
79   although the transmission dynamics have yet to...  
161  there is limited scientific evidence on the se...  
109  cdc is currently working on additional guidanc...  
127  remember a mask should only be used by health ...  
95   standard precautions should be followed when c...  


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Load a pre-trained multilingual model
model_name = "bert-base-multilingual-cased"  # or you can use "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Check if GPU is available and move model to GPU if so
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# Check if GPU is available and move model to GPU if so
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
def prepare_data(question, answer, max_length=512):
    inputs = tokenizer.encode_plus(
        question,
        answer,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )
    return inputs

# Prepare training data with start and end positions
def prepare_training_data(question, answer):
    inputs = prepare_data(question, answer)
    input_ids = inputs['input_ids'].squeeze()
    attention_mask = inputs['attention_mask'].squeeze()

    # Find the start and end positions of the answer in the context
    start_position = len(tokenizer.encode(question))  # Start right after the question
    end_position = start_position + len(tokenizer.encode(answer)) - 1  # Adjust for answer length

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'start_positions': torch.tensor(start_position),
        'end_positions': torch.tensor(end_position)
    }

train_encodings = [prepare_training_data(q, a) for q, a in zip(train_data['cleaned_question'], train_data['cleaned_answer'])]


Token indices sequence length is longer than the specified maximum sequence length for this model (708 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
model.train()
optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):  # Number of epochs
    for encodings in tqdm(train_encodings):
        optimizer.zero_grad()

        input_ids = encodings['input_ids'].unsqueeze(0).to(device)  # Add batch dimension
        attention_mask = encodings['attention_mask'].unsqueeze(0).to(device)  # Add batch dimension
        start_positions = encodings['start_positions'].unsqueeze(0).to(device)  # Add batch dimension
        end_positions = encodings['end_positions'].unsqueeze(0).to(device)  # Add batch dimension

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)

        # The loss is already calculated in the outputs
        loss = outputs.loss  # This is the total loss
        loss.backward()
        optimizer.step()

        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')  # Print the loss for monitoring

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  1%|          | 2/170 [00:01<02:00,  1.39it/s]

Epoch 1, Loss: 6.315886497497559
Epoch 1, Loss: 5.940020561218262


  2%|▏         | 4/170 [00:01<00:55,  3.01it/s]

Epoch 1, Loss: 5.931483268737793
Epoch 1, Loss: 5.735845565795898


  4%|▎         | 6/170 [00:02<00:36,  4.54it/s]

...

Epoch 3, Loss: 0.00042547742486931384
Epoch 3, Loss: 0.002495854627341032


 99%|█████████▉| 168/170 [00:24<00:00,  6.81it/s]

Epoch 3, Loss: 0.0019375223200768232
Epoch 3, Loss: 0.0001335593988187611


100%|██████████| 170/170 [00:24<00:00,  6.84it/s]

Epoch 3, Loss: 0.001581330201588571
Epoch 3, Loss: 0.000910636386834085


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# Set the model to evaluation mode
model.eval()

# Prepare the test data
test_encodings = [prepare_training_data(q, a) for q, a in zip(test_data['cleaned_question'], test_data['cleaned_answer'])]

all_preds = []
all_true = []

with torch.no_grad():
    for encodings in test_encodings:
        input_ids = encodings['input_ids'].unsqueeze(0).to(device)  # Add batch dimension
        attention_mask = encodings['attention_mask'].unsqueeze(0).to(device)  # Add batch dimension

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # Get the predicted start and end positions
        start_logits, end_logits = outputs.start_logits, outputs.end_logits

        start_pred = torch.argmax(start_logits, dim=-1).cpu().numpy()[0]
        end_pred = torch.argmax(end_logits, dim=-1).cpu().numpy()[0]

        all_preds.append((start_pred, end_pred))
        all_true.append((encodings['start_positions'].item(), encodings['end_positions'].item()))

# Calculate accuracy or F1 score
pred_start_positions, pred_end_positions = zip(*all_preds)
true_start_positions, true_end_positions = zip(*all_true)

accuracy = accuracy_score(true_start_positions, pred_start_positions)
f1 = f1_score(true_end_positions, pred_end_positions, average='weighted')

print(f'Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}')


Accuracy: 1.0000, F1 Score: 0.9767


In [ ]:
model.save_pretrained('/content/path')
tokenizer.save_pretrained('/content/path')

('/content/path/tokenizer_config.json',
 '/content/path/special_tokens_map.json',
 '/content/path/vocab.txt',
 '/content/path/added_tokens.json',
 '/content/path/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained('/content/path')
tokenizer = AutoTokenizer.from_pretrained('/content/path')


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/ask', methods=['POST'])
def ask():
    data = request.json
    question = data['question']
    context = data['context']  # This should be a relevant context for answering

    inputs = tokenizer.encode_plus(question, context, return_tensors="pt")
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    start_index = torch.argmax(outputs.start_logits)
    end_index = torch.argmax(outputs.end_logits)

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][start_index:end_index + 1]))

    return jsonify({'answer': answer})

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
import torch

def answer_question(question, context):
    # Encode the input question and context
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")

    # Move inputs to the same device as the model
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    # Get the model's prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the answer from the outputs
    start_logits, end_logits = outputs.start_logits, outputs.end_logits

    # Get the indices of the start and end of the predicted answer
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    # Decode the predicted answer
    answer_ids = inputs['input_ids'][0][start_index:end_index + 1]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True)

    return answer


Question: what are symptoms of covid?
Answer: A novel coronavirus is a new coronavirus that has not been previously identified. The virus causing coronavirus disease 2019 ( COVID - 19 ),


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load the tokenizer and the fine-tuned model
model_name = "/content/path"  # Replace with the actual path where your model is saved
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [ ]:
def get_answer(question):
    # Tokenize the input question
    inputs = tokenizer(question, padding=True, truncation=True, return_tensors="pt")
    input_ids = inputs['input_ids'].to(device)  # Move input_ids to the same device
    attention_mask = inputs['attention_mask'].to(device)  # Move attention_mask to the same device

    # Forward pass
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    # Get start and end logits
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Get the index of the start and end of the answer
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits) + 1  # +1 because the end index is inclusive

    # Decode the answer using the input tokens
    answer_tokens = input_ids[0][start_index:end_index]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    return answer.strip()

# Example questions to test the model
questions = [
    "What is a novel coronavirus?",
    "Why is the disease being called coronavirus disease 2019?",
    "How does the virus spread?"
]

# Get answers for each question
for question in questions:
    answer = get_answer(question)
    print(f"Question: {question}\nAnswer: {answer}\n")

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)